In [ ]:
import random
from time import sleep
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import json
import csv

In [ ]:
url = "https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie"

headers = {
    'accept': '*/*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 YaBrowser/23.3.2.806 Yowser/2.5 Safari/537.36'
}
req = requests.get(url, headers=headers)
src = req.text
# print(src)

In [ ]:
# сохраняем в файл
with open('index.html', 'w', encoding='utf-8') as file:
    file.write(src)

In [ ]:
# открытие файла, чтение и сохранение в переменную
with open('index.html', encoding='utf-8') as file:
    src = file.read()

soup = BeautifulSoup(src, 'lxml')

# первое - необходимо получить все ссылки на категории продуктов

all_products_hrefs = soup.find_all(class_='mzr-tc-group-item-href')

all_categories_dict = {}
for item in all_products_hrefs:
    item_text = item.text
    item_href = "https://health-diet.ru" + item.get('href')
    all_categories_dict[item_text] = item_href

# сохраняем словарь с категориями в файл json с отступом = 4
# ensure_ascii=False - не экранирует символы и помогает с кириллицей
with open('all_categores_dict.json', 'w', encoding='utf-8') as file:
    json.dump(all_categories_dict, file, indent=4, ensure_ascii=False)

In [ ]:
with open('all_categores_dict.json', encoding='utf-8') as file:
    all_categories = json.load(file)

In [51]:
# создаем цикл, на каждой итерации которого заходим в категорию, 
# берем данные и пишем в файл
iteration_count = int(len(all_categories)) - 1
count = 0
print(f'Всего итераций: {iteration_count}')
for category_name, category_href in all_categories.items():
    
    rep = [",", " ", "-", "'"]
    for item in rep:
        if item in category_name:
            category_name = category_name.replace(item, "_")
    req = requests.get(url = category_href, headers=headers)
    src = req.text
    with open(f"data/{count}_{category_name}.html", "w", encoding='utf-8') as file:
        file.write(src)
    
    with open(f"data/{count}_{category_name}.html", encoding='utf-8') as file:
        src = file.read()
    
    soup = BeautifulSoup(src, 'lxml')

    # проверка страницы на наличие таблицы с продуктами
    alert_blok = soup.find(class_="uk-alert-danger")
    if alert_blok is not None:
        continue

    # собираем заголовки таблицы
    table_head = soup.find(class_="mzr-tc-group-table").find("tr").find_all("th")
    product = table_head[0].text
    calories = table_head[1].text
    proteins = table_head[2].text
    fats = table_head[3].text
    carbohydrates = table_head[4].text
    
    with open(f"data/{count}_{category_name}.csv", "w", encoding='utf-8', newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                product,
                calories,
                proteins,
                fats,
                carbohydrates
            )
        )

    # собираем данные продуктов
    produts_data = soup.find(class_="mzr-tc-group-table").find("tbody").find_all("tr")

    product_info = []

    for item in produts_data:
        product_tds = item.find_all("td")

        title = product_tds[0].find("a").text
        calories = product_tds[1].text
        proteins = product_tds[2].text
        fats = product_tds[3].text
        carbohydrates = product_tds[4].text

        product_info.append(
            {
                "Title": title,
                "Calories": calories,
                'Proteins': proteins,
                'Fats': fats,
                'Carbohydrates': carbohydrates
            }
        )
        
        with open(f"data/{count}_{category_name}.csv", "a", encoding='utf-8', newline="") as file:
            writer = csv.writer(file)
            writer.writerow(
                (
                    title,
                    calories,
                    proteins,
                    fats,
                    carbohydrates
                )
            )

    with open(f'data/{category_name}.json', 'a', encoding='utf-8') as file:
        json.dump(product_info, file, indent=4, ensure_ascii=False)

    count += 1
    print(f"# Итерация {count}. {category_name} записан...")
    iteration_count -= 1

    if iteration_count == 0:
        print('Работа закончена')
        break

    print(f'Осталось итераций: {iteration_count}')
    sleep(random.randrange(2, 4))

Всего итераций: 53
# Итерация 1. Баранина_и_дичь записан...
Осталось итераций: 52
# Итерация 2. Бобовые записан...
Осталось итераций: 51
# Итерация 3. Вода_и_напитки записан...
Осталось итераций: 50
# Итерация 4. Говядина_и_телятина записан...
Осталось итераций: 49
# Итерация 5. Грибы записан...
Осталось итераций: 48
# Итерация 6. Детское_питание записан...
Осталось итераций: 47
# Итерация 7. Жиры_и_масла записан...
Осталось итераций: 46
# Итерация 8. Кондитерские_изделия записан...
Осталось итераций: 45
# Итерация 9. Крупы__мука__макароны записан...
Осталось итераций: 44
# Итерация 10. Молочные_продукты записан...
Осталось итераций: 43
# Итерация 11. Напитки_алкогольные записан...
Осталось итераций: 42
# Итерация 12. Овощи_и_зелень записан...
Осталось итераций: 41
# Итерация 13. Орехи_и_семена записан...
Осталось итераций: 40
# Итерация 14. Полуфабрикаты записан...
Осталось итераций: 39
# Итерация 15. Птица записан...
Осталось итераций: 38
# Итерация 16. Ресторанная_еда записан...
Ост